<a href="https://colab.research.google.com/github/Son-github/DeepLearning_PyTroch/blob/main/DeepLearning_with_PyTorch9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **순환 신경망(Recurrent Neural Network, RNN)**

numpy를 이용해 RNN 구현

In [1]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 됨
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 됨
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬

In [2]:
print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐

[0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
Wx = np.random.random((hidden_size, input_size))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

In [5]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [7]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.99953719 0.99999397 0.99999518 0.99996293 0.99999673 0.99999605
  0.99987314 0.9999915 ]
 [0.99973871 0.99999474 0.99999648 0.99996752 0.99999806 0.99999814
  0.99988039 0.99999463]
 [0.99983598 0.99999678 0.99999849 0.99998083 0.99999886 0.9999979
  0.99989348 0.99999734]
 [0.99958214 0.99998918 0.99999505 0.99986803 0.99999537 0.99998668
  0.99977017 0.99999008]
 [0.99977694 0.99999713 0.99999774 0.99997639 0.99999875 0.99999855
  0.99991646 0.99999616]
 [0.99977681 0.99998667 0.99999583 0.9999322  0.99999703 0.99999589
  0.99971466 0.9999931 ]
 [0.9997284  0.99999452 0.99999681 0.99996585 0.99999783 0.99999705
  0.99987259 0.99999494]
 [0.9997513  0.99999629 0.99999757 0.99995783 0.99999838 0.99999705
  0.9998891  0.99999526]
 [0.99984462 0.99999892 0.99999904 0.99998821 0.99999942 0.999999
  0.99995179 0.9999981 ]
 [0.99870619 0.99998069 0.99997551 0.99985959 0.99998662 0.99999124
  0.99983494 0.99997191]]


PyTorch의 nn.RNN()을 이용해 구현

In [8]:
import torch
import torch.nn as nn

In [9]:
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

In [10]:
# (batch_size, time_steps, input_size) => 배치 크기는 1, 10번의 시점동안 5차원의 입력 벡터가 들어가도록 텐서를 정의
inputs = torch.Tensor(1, 10, 5)

In [11]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)
# 입력의 크기, 은닉 상태의 크기를 정의. batch_first=True를 통해서 입력 텐서의 첫번째 차원이 배치의 크기임을 알려줌.

In [12]:
outputs, _status = cell(inputs)
# 첫번째 리턴값은 모든 시점(timesteps)의 은닉 상태들, 두번째 리턴값은 마지막 시점(timestep)의 은닉 상태

In [13]:
print(outputs.shape) # 모든 time_step의 hidden_state
# 10번의 시점동안 8차원의 은닉상태가 출력

torch.Size([1, 10, 8])


In [14]:
print(_status.shape) # 최종 time-step의 hidden_state

torch.Size([1, 1, 8])


깊은 순환 신경망(Deep Recurrent Neural Network)

In [15]:
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True)

In [18]:
outputs, _status = cell(inputs)

In [19]:
print(outputs.shape) # 모든 time-step의 hidden_state

torch.Size([1, 10, 8])


In [20]:
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)

torch.Size([2, 1, 8])


양방향 순환 신경망(Bidirectional Recurrent Neural Networt)

In [22]:
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True, bidirectional=True)

In [23]:
outputs, _status = cell(inputs)

In [24]:
print(outputs.shape) # 단방향 RNN 셀 때 보다 은닉 상태의 크기의 값이 두 배가 됨

torch.Size([1, 10, 16])


In [25]:
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)
# (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)를 가짐.

torch.Size([4, 1, 8])


PyTorch의 nn.LSTM()

In [26]:
# nn.LSTM(input_dim, hidden_size, batch_first=True) 형식으로 씀

PyTorch의 nn.GRU()

In [ ]:
# nn.GRU(input_dim, hidden_size, batch_first=True) 형식으로 씀